This code processes EDA and event-related data from .csv files generated by BIOPAC or similar devices. It structures and analyzes this data by applying conditions to identify events, downsampling the data, and organizing it for further analysis.  

Digital Channels: BIOPAC uses a set of digital input channels (e.g., 8 channels, D0 to D7).
Bit Position: 
D0 = 2^0 = 1  
D1 = 2^1 = 2 
D2 = 2^2 = 4 
D3 = 2^3 = 8 
D4 = 2^4 = 16 
D5 = 2^5 = 32 
D6 = 2^6 = 64 
D7 = 2^7 = 128 

Stim is in channel 16 (stim channel) 
GSR - EDA100C	
CORR - EMG100C	
ECG - ECG100C	
Feedback Cable - CBLCFMA - Current Feed	
Stim - Custom, AMI / HLT - A16	
Digital (STP Input 0)	
Digital (STP Input 1)	
Digital (STP Input 2)	
Digital (STP Input 3)	
Digital (STP Input 4)	
Digital (STP Input 5)	
Digital (STP Input 6)	
Digital (STP Input 7) 


Event Code Notes (which digital channels): 
Flanker Stimulus appears: 128 (channel: 7)

Each countdown/run starts with either: 
An event_code_countdown_start or event_code_flanker_start 
After the start of the flanker trial - there are these event codes: event_code_flanker 
 
countdown_end - this is when the whole coundfown section (60 seconds) ends and the electric shock is about to start. 

Note: errors from prev. script. keep just in case. 
In distal_shock condition: 
--- mistake countdown end: 91 (channel: 0, 1, 3, 6)
prox stim 
------ mistake - - countdown end: 92 (channels: 2, 3, 6 )
/end 



In distal_shock condition: 
countdown start: 81 (channel: 0, 4, 6 )
countdown end: 91 (channel: 0, 1, 3, 4, 6) *** fixed 
flanker start: 4 (channel: 2)
flanker end: 5 (channels: 0, 2)

In proximal_shock condition:
countdown start: 82 (channel: 1, 4, 6 )
countdown end: 92 (channels: 2, 3, 4, 6 ) *** fixed 
flanker start: 8 (channel: 3)
flanker end: 9(channels: 0, 3)

In distal_light_stim condition:
countdown start: 84 (channels: 2, 4, 6)
countdown end: 94 (channels: 0, 1, 2, 3, 4, 6 )
flanker start: 32 (channel: 5) 
flanker end: 33 (channels: 0, 5) 
		 	
In proximal_light_stim condition: 
countdown start: 85	(channels: 0, 2, 4, 6) 
countdown end: 95 (channels: 0, 1, 2, 3, 4, 6)
flanker start: 64 (channel 6) 
flanker end: 65 (channels: 0, 6) 


- This code runs through each subject, then through each task file from that subject.
- It imports the raw data of the task file (but aleady be in csv format after running an acq_to_mat script). 
- Each columns is a biopac channel (analog and digital channel).  
- Each row represents a time point, with values captured based on the sampling rate (e.g., 2000 Hz = 2000 rows per second).
- When an event code appears that event code tiggers a quick blip in the digital channels (either one or a combination of digital channels (cheat sheet above)). 
- To extract specific parts of the block - find the rows that have specific event codes. 
- Then break the data into 4 countdowns (there were 4 countdowns per block)  
- Label those parts of the block 
- Downsample (every 20th row) to make it a lower temporal resolution 
- Repeat with each block 
- Concatenate all the selected parts of the data into one file 
- Export as text file 



In [7]:
# List of conditions for sub 3 (36 countdowns) - info to double check if this script matches up 
# import csv 

import pandas as pd
file_path = "/Users/nadezhdabarbashova/Library/CloudStorage/Dropbox/LEAP_Neuro_Lab/researchProjects/nadu/fmcc/pipeline_psychopyz_nadu/sub_3_conditions.csv"
df = pd.read_csv(file_path, index_col=None)
df.reset_index(drop=True, inplace=True)  # Reset the index and remove the old one
print(df)



# 6 and 0 - end code 

    blockNum  intervalNum time_condition  shock_cond
0          1            1       proximal       shock
1          1            2         distal       shock
2          1            3       proximal  light_stim
3          1            4         distal  light_stim
4          2            5       proximal  light_stim
5          2            6       proximal       shock
6          2            7         distal  light_stim
7          2            8         distal       shock
8          3            9       proximal       shock
9          3           10         distal       shock
10         3           11         distal  light_stim
11         3           12       proximal  light_stim
12         4           13       proximal  light_stim
13         4           14         distal  light_stim
14         4           15       proximal       shock
15         4           16         distal       shock
16         5           17       proximal  light_stim
17         5           18         distal      

In [ ]:
# first column is time 
# second col is EDA 
# third column is event 


# get a start code 
# then start +60 seconds 
# export as csv 
# new index 
# next start code + 60 seconds 

In [31]:
# Script for collapsing across digital channels
import pandas as pd
import os
import numpy as np

#the timing files we want to get are three columns  - timestamp,  event code, event name
#import csv that you created, select the EDA column (& HR)

# change these 
save_dir = "/Users/nadezhdabarbashova/Library/CloudStorage/Dropbox/LEAP_Neuro_Lab/researchProjects/nadu/fmcc/data/acq_data/EDA_processed"
rawdata = "/Users/nadezhdabarbashova/Library/CloudStorage/Dropbox/LEAP_Neuro_Lab/researchProjects/nadu/fmcc/data/acq_data/fmcc_csv"
IDs = ["03"]
 

#We need to record the start event code conditions for runs so that it can be used for the other analysis
subject = []
runli = []
startcode = []
runs = [8]
 

#Define a function that find the index of not none elements in the list
def find_non_none_indices(lst):
    indices = [i for i, element in enumerate(lst) if element != 'none']
    return indices

for ID in IDs:
    for run in runs:
        print(run)
        subject.append(ID)
        runli.append(run) # in the end you have info to make a spreadsheet 
        current_dir = rawdata + "/" + ID
        current_file = "fmcc_sub" + ID + "_task_000" + str(run) + ".csv"
        path = os.path.join(current_dir, current_file)

        tmp_df = pd.read_csv(path, header=None, delimiter=',') #your txt or csv file from acqknowledge
        #print(tmp_df.head())
        # Current columns = EDA, CORR, ECG, Feedback, Stim, ch1-8
        #Let's remove the COrr & Feedback columns

        data = (tmp_df #make a copy of the df and...
        .drop(index=0)  # remove the first row
        .drop(columns=[1, 3])  # drop columns 1 and 3 (Corr & Feedback)
        .reset_index(drop=True))  # reset row index
        data.head()
        
        print(data) #take a look at the original data

        #rename the channels to make sense
        data.columns = ['EDA', 'ECG', 'Stim', 'ch0', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7']# renaming the channels from 0 - 7 instead of 1 - 8 based on Nadu's event code convention
	
        #Get the start of the countdown - this will be the codes for countdown_start OR flanker_start:
        conditions = [
             ((data['ch0'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5)& (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #distal shock countdown start
            ((data['ch1'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch0'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #proximal shock countdown start
            ((data['ch2'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch1'] == 0) & (data['ch0'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #distal stim countdown start
            ((data['ch0'] == 5) & (data['ch2'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch1'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #proximal stim countdown start

#              ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 5)& (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & (data['ch6'] == 0 ) & (data['ch7'] == 0 )) | #distal shock flanker start
#              ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & (data['ch6'] == 0 ) & (data['ch7'] == 0 )) | #proximal shock flanker start
#              ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 5) & (data['ch6'] == 0) & (data['ch7'] == 0 )) | #distal stim flanker start
#              ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & (data['ch6'] == 5 ) & (data['ch7'] == 0 ))  #proximal stim flanker start
      ]
        
        values = ['start', 'start', 'start', 'start']
        
        data['start'] = np.select(conditions, values, default="none")

       
        # deleted start line of code 

    #Notes from Jingyi's code that I don't think we need
        #divide the dataframe into four sections, two encoding sections, two task sections
        # Find the index of the first occurrence of the value
        #ps: +20 to make sure all the event code for 211 is removed.
        #index = data['start'].tolist().index('start') + 40

        #index = data['start'].tolist().index('start') #this line finds the index of the first occurrence of 'start' in the 'start' column -  9033698

        #first get the start_li and downsample - this is downsampling the 'start' column only for now by getting every 20th value 
        Start_li = data['start'].tolist()[::20] # creates a new list of the 'start' column
        
        Start_num = find_non_none_indices(Start_li) # this finds the index of every 'start' occurrence 
        startcode.append(len(Start_num)) 

        #then, Jingyi has if/then statements where she looks at the length of that Start_num list to see how many occurrences there were and she differentially works on the data - I don't think we need to do this

        #Instead, we are going to get a list of all of the first-instances of start (non-consecutive) to get the start of every countdown
        #start_indices = []
        #prev_value = None
        
        # for index, value in enumerate(data["column_name"])
        
        # Iterate over the 'start' column and check for transitions
        #for i, value in enumerate(data['start']):
           # if value == 'start' and (prev_value != 'start'):  # First occurrence of 'start'
            #    start_indices.append(i)
           # prev_value = value
       # print(start_indices) # [0, 903698, 905675, 907669, 909689, 911659, 913688, 915685, 917661, 919679, 921671, 923673, 925694, 927669, 929669, 931663, 933667, 935662, 937669, 939679, 941671, 943660, 945674, 947686, 949692, 951683, 953685, 955675, 957680, 959672, 961679, 963669, 965666]
       # len(start_indices) # [33]

        

             

SyntaxError: invalid syntax (1633472226.py, line 63)

In [ ]:
# next, we can slice the df into the different task epochs - countdown & flanker - we can do this for the different conditions

        # let's first create new columns that define the condition & timing:
        condition_conditions = [
            # Distal shock conditions
            ((data['ch0'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & 
            (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & 
            (data['ch5'] == 0) & (data['ch7'] == 0)), # distal shock countdown start 

            ((data['ch0'] == 5) & (data['ch1'] == 5) & (data['ch3'] == 5) & 
             (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch2'] == 0) & 
             (data['ch5'] == 0) & (data['ch7'] == 0)),  # distal shock countdown end *** fixed: channel 0, 1, 3, 4, 6

            ((data['ch2'] == 5) & (data['ch0'] == 0) & (data['ch1'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal shock flanker start 

            ((data['ch2'] == 5) & (data['ch0'] == 5) & (data['ch1'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal shock flanker end 

            # Proximal shock conditions
            ((data['ch0'] == 0) & (data['ch1'] == 5) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal shock countdown start 

            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 5) & 
            (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal shock countdown end 

            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # proximal shock flanker start 

            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # proximal shock flanker end 

            # Distal stim conditions
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 5) & 
            (data['ch3'] == 0) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # distal stim countdown start 

            ((data['ch0'] == 0) & (data['ch1'] == 5) & (data['ch2'] == 5) & 
            (data['ch3'] == 5) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # distal stim countdown end 

            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 5) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal stim flanker start 

            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 5) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal stim flanker end 

            # Proximal stim conditions
            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 5) & 
            (data['ch3'] == 0) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal stim countdown start

             ((data['ch2'] == 5) & (data['ch3'] == 5) & (data['ch4'] == 5) & 
             (data['ch6'] == 5) & (data['ch0'] == 0) & (data['ch1'] == 0) & 
             (data['ch5'] == 0) & (data['ch7'] == 0)),  # proximal stim countdown end *** fixed: channel 2, 3, 4, 6

            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal stim flanker start

            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)) # proximal stim flanker end 
        ]


        # Define corresponding condition values
        condition_values = [
            'distal shock', 'distal shock', 'distal shock', 'distal shock',
            'proximal shock', 'proximal shock', 'proximal shock', 'proximal shock',
            'distal stim', 'distal stim', 'distal stim', 'distal stim',
            'proximal stim', 'proximal stim', 'proximal stim', 'proximal stim'
        ]
        data['condition'] = np.select(condition_conditions, condition_values, default='none')
        
        epoch_values = [
            'countdown start', 'countdown end', 'flanker start', 'flanker end',
            'countdown start', 'countdown end', 'flanker start', 'flanker end',
            'countdown start', 'countdown end', 'flanker start', 'flanker end',
            'countdown start', 'countdown end', 'flanker start', 'flanker end'
        ]


        data['epoch'] = np.select(condition_conditions, epoch_values, default='none')
        event_values = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16]

        # 1 = distal shock countdown start
        # 2 = distal shock countdown end - we dont have this
        # 3 = distal shock flanker start
        # 4 = distal shock flanker end
        # 5 = proximal shock countdown start
        # 6 = proximal shock countdown end - we dont have this 
        # 7 = proximal shock flanker start
        # 8 = proximal shock flanker end
        # 9 = distal stim countdown start
        # 10 = distal stim countdown end
        # 11 = distal stim flanker start
        # 12 = distal stim flanker end
        # 13 = proximal stim countdown start
        # 14 = proximal stim countdown end
        # 15 = proximal stim flanker start
        # 16 = proximal stim flanker end 

        #then add a new column to our data frame based on these conditions
        data['Event'] = np.select(condition_conditions, event_values)
        
        #remove columns to clean up the df
        columns_to_remove = ['ch0', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7']
        data_sub = data.drop(columns=columns_to_remove)
        
        #print(data_sub) 

        # Construct the file name for the downsampled data
        test_file = os.path.join(save_dir, f"test_label_{ID}_run{run + 1}.csv")

        # Save the downsampled data as a CSV file
        data_sub.to_csv(test_file, index=False)
        print("saved as csv")

In [11]:
#data_downsampled = data_sub[::20].reset_index(drop=True)



        # lets also get a list of indices of the first instance of an event (non-consecutive)
        event_indices = data_sub[data_sub['Event'] != data_sub['Event'].shift(1)].index.tolist()
        event_indices = [i for i in event_indices if data_sub.loc[i, 'Event'] != 0]
        print(event_indices)
        
        # [4713, 6678, 8683, 10695, 12710, 14691, 16698, 18684, 20719, 22689, 24701, 26681, 28697, 30685, 32686, 34717, 36702, 38696, 40705, 42685, 44689, 46706, 48690, 50697, 52700, 54687, 56689, 58693, 60686, 62684, 64689, 66700, 124665, 151289, 153269, 153332, 213351, 213402, 215424, 217429, 219403, 221422, 223416, 225412, 227401, 229417, 231421, 233425, 235424, 237423, 239416, 241449, 243419, 245413, 247411, 249416, 251426, 253417, 255406, 257407, 259409, 261411, 263421, 265409, 267424, 269403, 271287, 271364, 297844, 299831, 359818, 359880, 361896, 363890, 365886, 367869, 369872, 371896, 373892, 375904, 377876, 379871, 381876, 383891, 385884, 387873, 389884, 391871, 393893, 395885, 397877, 399879, 401881, 403878, 405892, 407899, 409880, 411899, 413869, 415893, 417875, 417948, 418173, 441798, 443801, 445815, 447807, 449836, 451794, 453804, 455807, 457805, 459795, 461797, 463801, 465806, 467790, 469790, 471809, 473808, 475804, 477797, 479805, 481812, 483816, 485809, 487805, 489795, 491812, 493788, 495817, 497805, 499788, 501789, 503824, 561773, 562005]
        # these include 'countdown start' for every number in the countdown

        # preview a slice of the df
        print(data_sub.iloc[4710:4750])


        # let's slice the events from the beginning of one 'start' to the beginning of the next 'start' 
        # so start indices = [4713, 6678, 8683, 10695, 12710, 14691, 16698, 18684, 20719, 22689, 24701, 26681, 28697, 30685, 32686, 34717, 36702, 38696, 40705, 42685, 44689, 46706, 48690, 50697, 52700, 54687, 56689, 58693, 60686, 62684, 64689, 66700, 151289, 153269, 153332, 213402, 215424, 217429, 219403, 221422, 223416, 225412, 227401, 229417, 231421, 233425, 235424, 237423, 239416, 241449, 243419, 245413, 247411, 249416, 251426, 253417, 255406, 257407, 259409, 261411, 263421, 265409, 267424, 269403, 271287, 297844, 299831, 359880, 361896, 363890, 365886, 367869, 369872, 371896, 373892, 375904, 377876, 379871, 381876, 383891, 385884, 387873, 389884, 391871, 393893, 395885, 397877, 399879, 401881, 403878, 405892, 407899, 409880, 411899, 413869, 415893, 417875, 441798, 443801, 445815, 447807, 449836, 451794, 453804, 455807, 457805, 459795, 461797, 463801, 465806, 467790, 469790, 471809, 473808, 475804, 477797, 479805, 481812, 483816, 485809, 487805, 489795, 491812, 493788, 495817, 497805, 499788, 501789, 503824]

        # then our slices will be : [0:903697, 903698 : 905674, 905675 : 907668,etc]

        # the unique events we have are 5  7  8  1  3  4  9 11 12 10 13 15 16 14

        slices = [(start_indices[i], start_indices[i + 1] - 1) for i in range(len(start_indices) - 1)]
        # Include the final slice
        slices.append((start_indices[-1], 'end')) 
        print(slices)

        #now we're going to loop through that list of the countdown start indices and block out where the events start
        #for x in start_indices:
           #print(x)
           # data_tmp = data_sub.iloc[x:]
            
        #reset index
        # data_tmp.reset_index(inplace=True, drop=True)
        #data_tmp.head()
        
        print("done")

In [26]:
runs = [2]
def find_non_none_indices(lst):
    indices = [i for i, element in enumerate(lst) if element != 'none']
    return indices

for ID in IDs:
    for run in runs:
        print(run)
        subject.append(ID)
        runli.append(run)
        current_dir = rawdata + "/" + ID
        current_file = "fmcc_sub" + ID + "_task_000" + str(run) + ".csv"
        path = os.path.join(current_dir, current_file)

        tmp_df = pd.read_csv(path, header=None, delimiter=',') #your txt or csv file from acqknowledge
        print(tmp_df.head())
        # Current columns = EDA, CORR, ECG, Feedback, Stim, ch1-8
        #Let's remove the COrr & Feedback columns

        data = (tmp_df #make a copy of the df and...
        .drop(index=0)  # remove the first row
        .drop(columns=[1, 3])  # drop columns 1 and 3 (Corr & Feedback)
        .reset_index(drop=True))  # reset row index
        data.head()
        #rename the channels to make sense
        data.columns = ['EDA', 'ECG', 'Stim', 'ch0', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7']# renaming the channels from 0 - 7 instead of 1 - 8 based on Nadu's event code convention
	
        #Get the start of the countdown - this will be the codes for countdown_start OR flanker_start:
        conditions = [
            ((data['ch0'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5)& (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #distal shock countdown start
            ((data['ch1'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch0'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #proximal shock countdown start
            ((data['ch2'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch1'] == 0) & (data['ch0'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #distal stim countdown start
            ((data['ch0'] == 5) & (data['ch2'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & (data['ch1'] == 0) & (data['ch3'] == 0) & (data['ch5'] == 0 ) & (data['ch7'] == 0 )) | #proximal stim countdown start

            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 5)& (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & (data['ch6'] == 0 ) & (data['ch7'] == 0 )) | #distal shock flanker start
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & (data['ch6'] == 0 ) & (data['ch7'] == 0 )) | #proximal shock flanker start
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 5) & (data['ch6'] == 0) & (data['ch7'] == 0 )) | #distal stim flanker start
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & (data['ch6'] == 5 ) & (data['ch7'] == 0 ))  #proximal stim flanker start
        ]
        values = ["start"]
        # add a new column to put the start point
        data['start'] = np.select(conditions, values, default="none")


    #Notes from Jingyi's code that I don't think we need
        #divide the dataframe into four sections, two encoding sections, two task sections
        # Find the index of the first occurrence of the value
        #ps: +20 to make sure all the event code for 211 is removed.
        #index = data['start'].tolist().index('start') + 40


        index = data['start'].tolist().index('start') #this line finds the index of the first occurrence of 'start' in the 'start' column -  9033698

        #first get the start_li and downsample - this is downsampling the 'start' column only for now by getting every 20th value 
        Start_li = data['start'].tolist()[::20] # creates a new list of the 'start' column
        Start_num = find_non_none_indices (Start_li) # this finds the index of every 'start' occurrence 
        startcode.append(len(Start_num)) 

        #then, Jingyi has if/then statements where she looks at the length of that Start_num list to see how many occurrences there were and she differentially works on the data - I don't think we need to do this

        #Instead, we are going to get a list of all of the first-instances of start (non-consecutive) to get the start of every countdown
        start_indices = []
        prev_value = None
        # Iterate over the 'start' column and check for transitions
        for i, value in enumerate(data['start']):
            if value == 'start' and (prev_value != 'start'):  # First occurrence of 'start'
                start_indices.append(i)
            prev_value = value
        print(start_indices) # [0, 903698, 905675, 907669, 909689, 911659, 913688, 915685, 917661, 919679, 921671, 923673, 925694, 927669, 929669, 931663, 933667, 935662, 937669, 939679, 941671, 943660, 945674, 947686, 949692, 951683, 953685, 955675, 957680, 959672, 961679, 963669, 965666]
        len(start_indices) # [33]

        # next, we can slice the df into the different task epochs - countdown & flanker - we can do this for the different conditions

        # let's first create new columns that define the condition & timing:
        condition_conditions = [
            # Distal shock conditions
            ((data['ch0'] == 5) & (data['ch4'] == 5) & (data['ch6'] == 5) & 
            (data['ch1'] == 0) & (data['ch2'] == 0) & (data['ch3'] == 0) & 
            (data['ch5'] == 0) & (data['ch7'] == 0)), # distal shock countdown start 
            ((data['ch0'] == 5) & (data['ch1'] == 5) & (data['ch3'] == 5) & 
            (data['ch6'] == 5) & (data['ch2'] == 0) & (data['ch4'] == 0) & 
            (data['ch5'] == 0) & (data['ch7'] == 0)), # distal shock countdown end 
            ((data['ch2'] == 5) & (data['ch0'] == 0) & (data['ch1'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal shock flanker start 
            ((data['ch2'] == 5) & (data['ch0'] == 5) & (data['ch1'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal shock flanker end 

            # Proximal shock conditions
            ((data['ch0'] == 0) & (data['ch1'] == 5) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal shock countdown start 
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 5) & 
            (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal shock countdown end 
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # proximal shock flanker start 
            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 5) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # proximal shock flanker end 

            # Distal stim conditions
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 5) & 
            (data['ch3'] == 0) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # distal stim countdown start 
            ((data['ch0'] == 0) & (data['ch1'] == 5) & (data['ch2'] == 5) & 
            (data['ch3'] == 5) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # distal stim countdown end 
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 5) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal stim flanker start 
            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 5) & 
            (data['ch6'] == 0) & (data['ch7'] == 0)), # distal stim flanker end 

            # Proximal stim conditions
            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 5) & 
            (data['ch3'] == 0) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal stim countdown start
            ((data['ch0'] == 5) & (data['ch1'] == 5) & (data['ch2'] == 5) & 
            (data['ch3'] == 5) & (data['ch4'] == 5) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal stim countdown end
            ((data['ch0'] == 0) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)), # proximal stim flanker start
            ((data['ch0'] == 5) & (data['ch1'] == 0) & (data['ch2'] == 0) & 
            (data['ch3'] == 0) & (data['ch4'] == 0) & (data['ch5'] == 0) & 
            (data['ch6'] == 5) & (data['ch7'] == 0)) # proximal stim flanker end 
        ]

        # Define corresponding condition values
        condition_values = [
            'distal shock', 'distal shock', 'distal shock', 'distal shock',
            'proximal shock', 'proximal shock', 'proximal shock', 'proximal shock',
            'distal stim', 'distal stim', 'distal stim', 'distal stim',
            'proximal stim', 'proximal stim', 'proximal stim', 'proximal stim'
        ]
        data['condition'] = np.select(condition_conditions, condition_values, default='none')
        epoch_values = [
            'countdown start', 'countdown end', 'flanker start', 'flanker end',
            'countdown start', 'countdown end', 'flanker start', 'flanker end',
            'countdown start', 'countdown end', 'flanker start', 'flanker end',
            'countdown start', 'countdown end', 'flanker start', 'flanker end'
        ]
        data['epoch'] = np.select(condition_conditions, epoch_values, default='none')
        event_values = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16]
        # 1 = distal shock countdown start
        # 2 = distal shock countdown end - we dont have this
        # 3 = distal shock flanker start
        # 4 = distal shock flanker end
        # 5 = proximal shock countdown start
        # 6 = proximal shock countdown end - we dont have this 
        # 7 = proximal shock flanker start
        # 8 = proximal shock flanker end
        # 9 = distal stim countdown start
        # 10 = distal stim countdown end
        # 11 = distal stim flanker start
        # 12 = distal stim flanker end
        # 13 = proximal stim countdown start
        # 14 = proximal stim countdown end
        # 15 = proximal stim flanker start
        # 16 = proximal stim flanker end 

        #then add a new column to our data frame based on these conditions
        data['Event'] = np.select(condition_conditions, event_values)
        
        #remove columns to clean up the df
        columns_to_remove = ['ch0', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7']
        data_sub = data.drop(columns=columns_to_remove)

        # lets also get a list of indices of the first instance of an event (non-consecutive)
        event_indices = data_sub[data_sub['Event'] != data_sub['Event'].shift(1)].index.tolist()
        event_indices = [i for i in event_indices if data_sub.loc[i, 'Event'] != 0]
        print(event_indices)
        # [4713, 6678, 8683, 10695, 12710, 14691, 16698, 18684, 20719, 22689, 24701, 26681, 28697, 30685, 32686, 34717, 36702, 38696, 40705, 42685, 44689, 46706, 48690, 50697, 52700, 54687, 56689, 58693, 60686, 62684, 64689, 66700, 124665, 151289, 153269, 153332, 213351, 213402, 215424, 217429, 219403, 221422, 223416, 225412, 227401, 229417, 231421, 233425, 235424, 237423, 239416, 241449, 243419, 245413, 247411, 249416, 251426, 253417, 255406, 257407, 259409, 261411, 263421, 265409, 267424, 269403, 271287, 271364, 297844, 299831, 359818, 359880, 361896, 363890, 365886, 367869, 369872, 371896, 373892, 375904, 377876, 379871, 381876, 383891, 385884, 387873, 389884, 391871, 393893, 395885, 397877, 399879, 401881, 403878, 405892, 407899, 409880, 411899, 413869, 415893, 417875, 417948, 418173, 441798, 443801, 445815, 447807, 449836, 451794, 453804, 455807, 457805, 459795, 461797, 463801, 465806, 467790, 469790, 471809, 473808, 475804, 477797, 479805, 481812, 483816, 485809, 487805, 489795, 491812, 493788, 495817, 497805, 499788, 501789, 503824, 561773, 562005]
        # these include 'countdown start' for every number in the countdown

        # preview a slice of the df
        print(data_sub.iloc[4710:4750])


        # let's slice the events from the beginning of one 'start' to the beginning of the next 'start' 
        # so start indices = [4713, 6678, 8683, 10695, 12710, 14691, 16698, 18684, 20719, 22689, 24701, 26681, 28697, 30685, 32686, 34717, 36702, 38696, 40705, 42685, 44689, 46706, 48690, 50697, 52700, 54687, 56689, 58693, 60686, 62684, 64689, 66700, 151289, 153269, 153332, 213402, 215424, 217429, 219403, 221422, 223416, 225412, 227401, 229417, 231421, 233425, 235424, 237423, 239416, 241449, 243419, 245413, 247411, 249416, 251426, 253417, 255406, 257407, 259409, 261411, 263421, 265409, 267424, 269403, 271287, 297844, 299831, 359880, 361896, 363890, 365886, 367869, 369872, 371896, 373892, 375904, 377876, 379871, 381876, 383891, 385884, 387873, 389884, 391871, 393893, 395885, 397877, 399879, 401881, 403878, 405892, 407899, 409880, 411899, 413869, 415893, 417875, 441798, 443801, 445815, 447807, 449836, 451794, 453804, 455807, 457805, 459795, 461797, 463801, 465806, 467790, 469790, 471809, 473808, 475804, 477797, 479805, 481812, 483816, 485809, 487805, 489795, 491812, 493788, 495817, 497805, 499788, 501789, 503824]

        # then our slices will be : [0:903697, 903698 : 905674, 905675 : 907668,etc]

        # the unique events we have are 5  7  8  1  3  4  9 11 12 10 13 15 16 14

        slices = [(start_indices[i], start_indices[i + 1] - 1) for i in range(len(start_indices) - 1)]
        # Include the final slice
        slices.append((start_indices[-1], 'end')) 
        print(slices)

2
         0         1         2         3         4    5    6    7    8     9   \
0  1.000000  2.000000  3.000000  4.000000  5.000000  6.0  7.0  8.0  9.0  10.0   
1  2.540402 -0.012207  0.643921  0.314331 -0.209346  0.0  0.0  0.0  0.0   0.0   
2  2.537350 -0.009155  0.650330  0.299072 -0.207210  0.0  0.0  0.0  0.0   0.0   
3  2.541928  0.022888  0.656738  0.332642 -0.206600  0.0  0.0  0.0  0.0   0.0   
4  2.540402  0.051880  0.662537  0.308228 -0.209652  0.0  0.0  0.0  0.0   0.0   

     10    11    12  
0  11.0  12.0  13.0  
1   0.0   0.0   0.0  
2   0.0   0.0   0.0  
3   0.0   0.0   0.0  
4   0.0   0.0   0.0  


ValueError: 'start' is not in list